In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Text summarization

In [2]:
!pip install PyPDF2==2.10.8
!pip install transformers==4.22.0
!pip install datasets==2.4.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 217 kB 32.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.9 MB 30.2 MB/s 
     |████████████████████████████████| 120 kB 64.8 MB/s 
     |████████████████████████████████| 6.6 MB 56.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 28.1 MB/s 
     |████████████████████████████████| 212 kB 70.9 MB/s 
     |████████████████████████████████| 115 kB 74.5 MB/s 
     |████████████████████████████████| 127 kB 75.7 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
from PyPDF2 import PdfReader
from transformers import pipeline

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [4]:
def remove_unnecessay_lines(text):

  final_text = ""

  for line in text.split('\n'):
    if len(line.split(' ')) > 1:
      final_text += line
  return final_text

In [5]:
def extract_summary(reader_obj, summary_obj):
  count = reader.numPages
  for i in range(count):
    page = reader.getPage(i)
    text = page.extract_text()
    final_text = remove_unnecessay_lines(text)
    summary_ =  summarizer(final_text, max_length=500, min_length=30, do_sample=False)[0]['summary_text']
    return summary_

In [6]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/1.80k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

In [7]:
def extract_summary(reader_obj, summary_obj):
  count = reader.numPages
  for i in range(count):
      page = reader.getPage(i)
      text = page.extract_text()
      final_text = remove_unnecessay_lines(text)
  summary_ =  summarizer(final_text, max_length = 0.2 * len(final_text), min_length= 30, do_sample=False)[0]['summary_text']
  return summary_

In [8]:
reader = PdfReader("/content/drive/MyDrive/stock_prediction_system/Data/q1-2023.pdf")
summary = []
summary.append(extract_summary(reader, summarizer))

In [9]:
summary

[' The American Depositary Shares issued under the ADS program of the company were listed on the NASDAQ National Market in the US on March 11, 1999 . The Company began trading of its ADSs on NYSE Euronext’s (NYX) London and Paris markets on February 20, 2013 . The number of ADSs outstanding as on June 30,2022 was 59,73,90,828. The percentage of volume traded to the total float was 114.68%.']

Classification

In [10]:
import numpy as np
import pandas as pd 
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments, AutoModelForSequenceClassification
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [11]:
import torch
import transformers
torch.__version__, transformers.__version__

('1.12.1+cu113', '4.22.0')

In [12]:
torch.cuda.is_available()

True

In [13]:
df = pd.read_csv('/content/drive/MyDrive/stock_prediction_system/dataset/finbert_clean.csv',nrows=10000) ## use your own customized dataset
df.head()

,id,sentence,prediciton,sentiment_score
0,0,b of a securities maintains neutral on agilent...,positive,0.568038
1,1,cfra maintains hold on agilent technologies lo...,negative,-0.335676
2,2,wells fargo maintains overweight on agilent te...,positive,0.793402
3,3,svb leerink maintains outperform on agilent te...,positive,0.701838
4,4,agilent technologies q2 adj eps 071 beats 061 ...,negative,-0.119871


In [14]:
df['prediciton'].unique()

array(['positive', 'negative', 'neutral'], dtype=object)

In [15]:
df['label'] = df['prediciton'].replace({'positive':1, 'negative':2, 'neutral':0})

In [16]:
df.drop(['prediciton','sentiment_score','id'],axis='columns',inplace=True)

In [17]:
df.head()

,sentence,label
0,b of a securities maintains neutral on agilent...,1
1,cfra maintains hold on agilent technologies lo...,2
2,wells fargo maintains overweight on agilent te...,1
3,svb leerink maintains outperform on agilent te...,1
4,agilent technologies q2 adj eps 071 beats 061 ...,2


In [18]:
df = df.dropna(subset=['sentence', 'label'])

In [19]:
df_train, df_test, = train_test_split(df, stratify=df['label'], test_size=0.1, random_state=42)
df_train, df_val = train_test_split(df_train, stratify=df_train['label'],test_size=0.1, random_state=42)
print(df_train.shape, df_test.shape, df_val.shape)

(8100, 2) (1000, 2) (900, 2)


In [ ]:
model = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-pretrain',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')

Downloading:   0%|          | 0.00/359 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/442M [00:00<?, ?B/s]

In [ ]:
dataset_train = Dataset.from_pandas(df_train)
dataset_val = Dataset.from_pandas(df_val)
dataset_test = Dataset.from_pandas(df_test)

dataset_train = dataset_train.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_val = dataset_val.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length', max_length=128), batched=True)
dataset_test = dataset_test.map(lambda e: tokenizer(e['sentence'], truncation=True, padding='max_length' , max_length=128), batched=True)

dataset_train.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_val.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
dataset_test.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])

In [22]:
def compute_metrics(eval_pred):
    predictions, label = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {'accuracy' : accuracy_score(predictions,label)}

args = TrainingArguments(
        output_dir = '/content/drive/MyDrive/stock_prediction_system/temp/',
        evaluation_strategy = 'epoch',
        save_strategy = 'epoch',
        learning_rate=2e-5,
        per_device_train_batch_size=32,
        per_device_eval_batch_size=32,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model='accuracy',
)

trainer = Trainer(
        model=model,                         # the instantiated 🤗 Transformers model to be trained
        args=args,                  # training arguments, defined above
        train_dataset=dataset_train,         # training dataset
        eval_dataset=dataset_val,            # evaluation dataset
        compute_metrics=compute_metrics
)

  

In [23]:
trainer.train() 

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, sentence. If __index_level_0__, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 8100
  Num Epochs = 5
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1270


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.421006,0.843333
2,0.405100,0.422754,0.841111
3,0.405100,0.441444,0.844444
4,0.150400,0.532759,0.838889
5,0.150400,0.560726,0.847778


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, sentence. If __index_level_0__, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 900
  Batch size = 32
Saving model checkpoint to /content/drive/MyDrive/stock_prediction_system/temp/checkpoint-254
Configuration saved in /content/drive/MyDrive/stock_prediction_system/temp/checkpoint-254/config.json
Model weights saved in /content/drive/MyDrive/stock_prediction_system/temp/checkpoint-254/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, sentence. If __index_level_0__, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Eval

TrainOutput(global_step=1270, training_loss=0.23548082667072928, metrics={'train_runtime': 912.3664, 'train_samples_per_second': 44.39, 'train_steps_per_second': 1.392, 'total_flos': 2664023354496000.0, 'train_loss': 0.23548082667072928, 'epoch': 5.0})

In [24]:
model.eval()
trainer.predict(dataset_test).metrics

The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, sentence. If __index_level_0__, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 32


{'test_loss': 0.5030061602592468,
 'test_accuracy': 0.872,
 'test_runtime': 7.0935,
 'test_samples_per_second': 140.973,
 'test_steps_per_second': 4.511}

In [25]:
trainer.save_model("/content/drive/MyDrive/stock_prediction_system/model/Trained model")

Saving model checkpoint to /content/drive/MyDrive/stock_prediction_system/model/Trained model
Configuration saved in /content/drive/MyDrive/stock_prediction_system/model/Trained model/config.json
Model weights saved in /content/drive/MyDrive/stock_prediction_system/model/Trained model/pytorch_model.bin


In [26]:
model_loaded1 = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/stock_prediction_system/model/Trained model")

loading configuration file /content/drive/MyDrive/stock_prediction_system/model/Trained model/config.json
Model config BertConfig {
  "_name_or_path": "/content/drive/MyDrive/stock_prediction_system/model/Trained model",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.22.0",
  "type_vocab_size": 2,
  "use_cache": true,

In [27]:
trainer1 = Trainer(model = model_loaded1)
test_results = trainer1.predict(dataset_test)

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: __index_level_0__, sentence. If __index_level_0__, sentence are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1000
  Batch size = 8


In [28]:
np.argmax(test_results[0],axis=1)

array([2, 0, 2, 1, 2, 2, 1, 2, 0, 0, 2, 2, 2, 1, 2, 0, 2, 1, 2, 2, 1, 1,
       2, 1, 1, 2, 0, 2, 2, 0, 2, 2, 0, 2, 2, 0, 1, 1, 0, 1, 1, 2, 1, 2,
       1, 2, 2, 1, 0, 1, 0, 1, 1, 1, 2, 2, 0, 1, 0, 0, 1, 1, 0, 1, 1, 2,
       2, 0, 2, 1, 0, 0, 2, 1, 2, 1, 1, 2, 0, 0, 1, 0, 0, 0, 2, 2, 2, 1,
       0, 2, 0, 2, 1, 2, 0, 1, 1, 1, 2, 2, 1, 1, 2, 1, 2, 2, 2, 0, 0, 2,
       2, 0, 1, 2, 2, 2, 2, 2, 0, 2, 1, 1, 0, 1, 2, 2, 0, 0, 2, 1, 1, 0,
       0, 0, 0, 2, 1, 0, 1, 0, 2, 2, 0, 0, 1, 0, 2, 0, 0, 0, 1, 2, 0, 1,
       2, 1, 1, 0, 1, 1, 1, 2, 0, 1, 1, 2, 0, 1, 1, 2, 1, 1, 1, 2, 0, 2,
       1, 1, 0, 1, 0, 2, 1, 1, 0, 1, 1, 0, 1, 0, 2, 2, 0, 2, 2, 1, 2, 2,
       0, 1, 1, 2, 2, 2, 1, 2, 0, 1, 0, 0, 0, 0, 2, 1, 1, 0, 0, 0, 1, 1,
       2, 1, 1, 1, 1, 1, 2, 1, 2, 0, 2, 1, 0, 1, 0, 2, 2, 1, 0, 2, 0, 2,
       0, 2, 1, 2, 1, 2, 0, 0, 1, 1, 0, 2, 2, 1, 0, 2, 1, 2, 2, 1, 2, 0,
       1, 1, 2, 0, 1, 0, 2, 1, 1, 2, 2, 1, 0, 1, 2, 2, 0, 2, 1, 2, 1, 1,
       2, 0, 2, 2, 2, 1, 2, 1, 0, 1, 1, 0, 1, 0, 1,

Prediction

In [29]:
!pip install PyPDF2==2.10.8
!pip install transformers==4.22.0
!pip install datasets==2.4.0
import numpy as np
import pandas as pd 
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments, AutoModelForSequenceClassification
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import torch
import transformers
torch.__version__, transformers.__version__
torch.cuda.is_available()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


True

In [30]:
def testing(text):
  data=pd.DataFrame()
  data['text'] = text
  print(data)
  label_id = {0:'neutral', 1:'positive',2:'negative'}
  tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-pretrain')
  model_loaded = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/stock_prediction_system/model/Trained model")
  datatesting = Dataset.from_pandas(data)
  datatesting = datatesting.map(lambda e: tokenizer(e['text'], truncation=True, padding='max_length', max_length=128), batched=True)
  datatesting.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask'])
  print(datatesting)
  trainer_model = Trainer(model = model_loaded)
  testing_results = trainer_model.predict(datatesting)
  pred = testing_results
  pred_class = np.argmax(pred[0],axis=1)
  print(pred_class)
  pred_label=[]
  if pred_class==0:
    pred_label.append('Neutral')
  elif pred_class==1:
    pred_label.append('Positive')
  else:
    pred_label.append('Negative')
  return pred_label

In [31]:
text = ['Tata Motors Group Results -Q4 FY22  May 12, 2022 (CONSOLIDATED NUMBERS , IND AS) Finance costs increased by ₹ 1, 215Cr to  9,312Cr during FY’22 due to higher gross borrowings . Free cash flow (automotive) in the year, was negative at �\xa09.5 Cr (as compared to  positive  �\xa05.3 Cr in  FY 21), primarily due to  working capital impact of ⁹9.6 Cr .']
predictions = testing(text)

                                                text
0  Tata Motors Group Results -Q4 FY22  May 12, 20...


loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--yiyanghkust--finbert-pretrain/snapshots/5b0dae12fea8ca5b3f256267ebe4e21786f3cfe5/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--yiyanghkust--finbert-pretrain/snapshots/5b0dae12fea8ca5b3f256267ebe4e21786f3cfe5/config.json
Model config BertConfig {
  "_name_or_path": "yiyanghkust/finbert-pretrain",
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  

  0%|          | 0/1 [00:00<?, ?ba/s]

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


Dataset({
    features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 1
})


The following columns in the test set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8


[2]


In [32]:
print(predictions[0])

Negative
